In [2]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler



In [ ]:
# Đọc file CSV
file_path = 'product_details.csv'  # Đường dẫn file
data = pd.read_csv(file_path)

# Hiển thị thông tin tổng quan về dữ liệu
print("Thông tin tổng quan về dữ liệu:")
print(data.info())
display(data.head())

# Bước 1: Kiểm tra và xử lý giá trị trùng lặp
print("\nSố lượng giá trị trùng lặp trước khi xử lý:", data.duplicated().sum())
data = data.drop_duplicates()
print("Số lượng giá trị trùng lặp sau khi xử lý:", data.duplicated().sum())

# Bước 2: Kiểm tra và xử lý giá trị thiếu
missing_values = data.isnull().sum()
print("\nSố lượng giá trị thiếu trong mỗi cột:")
print(missing_values)

# Xử lý giá trị thiếu:
# - Chuyển đổi 'price' và 'sold' sang dạng số
data['price'] = pd.to_numeric(data['price'].str.replace('₫', '').str.replace(',', ''), errors='coerce')
data['sold'] = data['sold'].str.replace(' sold', '').str.replace('K', 'e3')
data['sold'] = pd.to_numeric(data['sold'], errors='coerce')

# - Điền giá trị thiếu: trung bình cho 'price', và 0 cho 'sold'
data['price'].fillna(data['price'].mean(), inplace=True)
data['sold'].fillna(0, inplace=True)

# Thêm cột 'estimated_revenue' (doanh thu ước tính)
data['estimated_revenue'] = data['price'] * data['sold']

# Kiểm tra lại dữ liệu sau khi xử lý giá trị thiếu
print("\nDữ liệu sau khi xử lý giá trị thiếu:")
print(data.isnull().sum())

# Bước 3: Chuẩn hóa dữ liệu
# Chỉ chuẩn hóa các cột số
numeric_cols = ['price', 'sold', 'estimated_revenue']
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[numeric_cols])

# Chuyển dữ liệu đã chuẩn hóa thành DataFrame
scaled_df = pd.DataFrame(data_scaled, columns=[f"{col}_scaled" for col in numeric_cols])

# Kết hợp dữ liệu gốc và dữ liệu đã chuẩn hóa
final_data = pd.concat([data.reset_index(drop=True), scaled_df], axis=1)



# Bước 3: Lưu dữ liệu vào file CSV duy nhất
output_path = 'preprocessing.csv'
final_data.to_csv(output_path, index=False)
print(f"\nDữ liệu cuối cùng đã được lưu tại: {output_path}")



Thông tin tổng quan về dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1070 non-null   object
 1   store_name     1068 non-null   object
 2   rating         1070 non-null   object
 3   comment_count  1070 non-null   object
 4   price          1070 non-null   object
 5   sold           1070 non-null   object
 6   link           1070 non-null   object
dtypes: object(7)
memory usage: 58.6+ KB
None


,product_name,store_name,rating,comment_count,price,sold,link
0,"[HCM]Thẻ nhớ 64gb, Thẻ nhớ camera YOOSEE Micro...",Không tìm thấy cửa hàng,0,Không có đánh giá,"₫84,800",594 sold,https://www.lazada.vn/products/hcmthe-nho-64gb...
1,"[HCM]Thẻ nhớ 32gb, Thẻ nhớ camera YOOSEE Micro...",Không tìm thấy cửa hàng,0,Không có đánh giá,"₫66,000",3.8K sold,https://www.lazada.vn/products/hcmthe-nho-32gb...
2,Thẻ nhớ TF 16GB 32GB 64GB 128GB 256GB 512GB 1T...,Không tìm thấy cửa hàng,0,Không có đánh giá,"₫49,000",561 sold,https://www.lazada.vn/products/the-nho-tf-16gb...
3,Thẻ Nhớ Kioxia 32GB và Thẻ Nhớ Sandisk (64GB/1...,EZVIZ VN Authorized Store,5.0,750 đánh giá,"₫159,000",2.8K sold,https://www.lazada.vn/products/the-nho-kioxia-...
4,Thẻ Nhớ 32 gb ( Thẻ nhớ dành chuyên cho camera...,Không tìm thấy cửa hàng,0,Không có đánh giá,"₫25,000",96 sold,https://www.lazada.vn/products/the-nho-32-gb-t...



Số lượng giá trị trùng lặp trước khi xử lý: 34
Số lượng giá trị trùng lặp sau khi xử lý: 0

Số lượng giá trị thiếu trong mỗi cột:
product_name     0
store_name       2
rating           0
comment_count    0
price            0
sold             0
link             0
dtype: int64

Dữ liệu sau khi xử lý giá trị thiếu:
product_name         0
store_name           2
rating               0
comment_count        0
price                0
sold                 0
link                 0
estimated_revenue    0
dtype: int64

Dữ liệu cuối cùng sau xử lý và chuẩn hóa:


C:\Users\DELL\AppData\Local\Temp\ipykernel_2628\3694623583.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['price'].fillna(data['price'].mean(), inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_2628\3694623583.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

,product_name,store_name,rating,comment_count,price,sold,link,estimated_revenue,price_scaled,sold_scaled,estimated_revenue_scaled
0,"[HCM]Thẻ nhớ 64gb, Thẻ nhớ camera YOOSEE Micro...",Không tìm thấy cửa hàng,0,Không có đánh giá,84800.0,594.0,https://www.lazada.vn/products/hcmthe-nho-64gb...,50371200.0,-0.310662,-0.213757,-0.143959
1,"[HCM]Thẻ nhớ 32gb, Thẻ nhớ camera YOOSEE Micro...",Không tìm thấy cửa hàng,0,Không có đánh giá,66000.0,3800.0,https://www.lazada.vn/products/hcmthe-nho-32gb...,250800000.0,-0.322666,0.312776,-0.068376
2,Thẻ nhớ TF 16GB 32GB 64GB 128GB 256GB 512GB 1T...,Không tìm thấy cửa hàng,0,Không có đánh giá,49000.0,561.0,https://www.lazada.vn/products/the-nho-tf-16gb...,27489000.0,-0.333520,-0.219177,-0.152588
3,Thẻ Nhớ Kioxia 32GB và Thẻ Nhớ Sandisk (64GB/1...,EZVIZ VN Authorized Store,5.0,750 đánh giá,159000.0,2800.0,https://www.lazada.vn/products/the-nho-kioxia-...,445200000.0,-0.263285,0.148542,0.004933
4,Thẻ Nhớ 32 gb ( Thẻ nhớ dành chuyên cho camera...,Không tìm thấy cửa hàng,0,Không có đánh giá,25000.0,96.0,https://www.lazada.vn/products/the-nho-32-gb-t...,2400000.0,-0.348844,-0.295546,-0.162049



Dữ liệu cuối cùng đã được lưu tại: preprocessing.csv
